In [1]:
import pandas as pd
import openpyxl
from datetime import datetime, time

def categorizar_hora(hora_str):
    """
    Cambiar la variable hora a una variable categorica.
    Se divide en 3 rangos, respondiendo al huso horario CET (UTC+01:00):
    - 6:00 (6 AM) A 14:00 (2 PM) se considera Mañana, 
    - 14:00 (2 PM) a 20:00 (8 PM) se considera Tarde, y 
    - 20:00 (8 PM) a 6:00 (6 AM) se considera Noche.
    """
    if hora_str is not None:
        hora = datetime.strptime(hora_str, '%H:%M:%S').time()

        if time(6) <= hora < time(14):
            return 'Mañana'
        elif time(14) <= hora < time(21):
            return 'Tarde'
        elif time(21) <= hora < time(0):
            return 'Noche'
        else:
            return 'Noche'  # Incluye las horas entre medianoche y 6 am
        
# Cargar el archivo Excel con las caracteristicas extraidas para cada post
wb_post = openpyxl.load_workbook('info_post_sin_texto_etiquetas.xlsx')
hoja_post = wb_post.active
# hoja_post.cell(row=1, column=17, value='caracteristicas')
hoja_post.cell(row=1, column=23, value='hora')

for index, row in enumerate(hoja_post.iter_rows(min_row = 1, values_only = True)):
    # Transformar las columnas de texto y hashgtags en numericas
    descripcion = hoja_post.cell(row=index+2, column=15).value
    if descripcion is not None:
        hoja_post.cell(row=index+2, column=15, value=len(descripcion))

    hashgtags = str(hoja_post.cell(row=index+2, column=16).value)
    if hashgtags is not None:
        hoja_post.cell(row=index+2, column=16, value=len(hashgtags.split(';')))

    # Transformar la variable hora
    hora = hoja_post.cell(row=index+2, column=6).value
    if categorizar_hora(hora) == 'Mañana':
        hoja_post.cell(row=index+2, column=23, value='mañana')
    elif categorizar_hora(hora) == 'Tarde':
        hoja_post.cell(row=index+2, column=23, value='tarde')
    elif categorizar_hora(hora) == 'Noche':
        hoja_post.cell(row=index+2, column=23, value='noche')
    
    # Transformar la variable fecha
    if hoja_post.cell(row=index+2, column=5).value is not None:
        hoja_post.cell(row=index+2, column=24, value=hoja_post.cell(row=index+2, column=5).value.strftime('%A'))
                
hoja_post.delete_cols(6)
hoja_post.delete_cols(5)

wb_post.save("info_post_sin_texto_etiquetas_categorico.xlsx")

In [3]:
# Extraer las variables premios y num_post_año (correspondientes al usuario) para cada post
df_ganadores = pd.read_excel('C:TFG\\excels\\info_ganadores.xlsx')

# Crear el diccionario {usuario: [premios, post]}
dict_ganadores = df_ganadores.set_index('INSTAGRAM')[['PREMIOS', 'POSTS']].to_dict(orient='index')
dict_ganadores = {ganador: [fila['PREMIOS'], fila['POSTS']] for ganador, fila in dict_ganadores.items()}

df_post_etiquetas = pd.read_excel('info_post_sin_texto_etiquetas_categorico.xlsx')

# Añadir las columnas para premios y post
df_post_etiquetas['PREMIOS'] = 0
df_post_etiquetas['POSTS'] = 0

# Recorrer el DataFrame y actualizar las columnas
for index, row in df_post_etiquetas.iterrows():
    usuario = row['usuario']
    if usuario in dict_ganadores:
        df_post_etiquetas.at[index, 'PREMIOS'] = dict_ganadores[usuario][0]
        df_post_etiquetas.at[index, 'POSTS'] = dict_ganadores[usuario][1]

# Guardar el DataFrame actualizado en un nuevo archivo Excel
df_post_etiquetas.to_excel('info_post_sin_texto_etiquetas_categorico.xlsx', index=False)